In [1]:
import os, json

In [ ]:
####################
# Loading raw data #
####################
folder = 'raw/' # Folder with data_backend_tik_tok files + TikTok.json + voices.json
data = []
for file_name in os.listdir('raw/'):
    if 'data_backend' in file_name:
        print(file_name)
        with open(os.path.join(folder, file_name), 'r', encoding="utf8") as f:
            for line in f.readlines():
                data.append(json.loads(line))

print(len(data))
print('Done!')

In [ ]:
# Load enriched data
with open(os.path.join('enriched', 'TikTok.json'), 'r') as f:
    enriched_data = json.load(f)
print(len(enriched_data))
print('Done!')

In [ ]:
# Transform into dict of enriched data
enriched_data = {enrich_data['nap']['id']: enrich_data for enrich_data in enriched_data}
print('Done!')

In [ ]:
print(len(enriched_data), 'distinct nap id')
print('Done!')

In [ ]:
# Enrich raw_data with brand and photo_text_enrichment when available
for raw_data in data:
    nap_id = raw_data['id']
    if nap_id in enriched_data.keys():
        # Add brand key
        raw_data['brand'] = enriched_data[nap_id]['brand']
        raw_data['photo_text_enrichment'] = enriched_data[nap_id]['photo_text_enrichment']
print('Done!')

In [ ]:
# Then load voices into a simple list
voices = []
with open(os.path.join('voices', 'voices.json'), 'r', encoding="utf8") as f:
    for line in f.readlines():
        voices.append(json.loads(line))
print(len(voices))
print('Done!')

In [ ]:
#convert voices into a dict of voices

tiktok_voices = {}
for voice in voices:
    
    # Filter with 'celebrity', 'influencer'
    
    #### NOW WITH ALL CATEGORIES ####
    #if voice['type'] not in ['celebrity', 'influencer']:
    #    continue
    ##################################        
    for source in voice['sources']:
        
        if source['channel'] == 'tiktok':
            
            # Copy parent infos to source
            for field in ['country', 'name', 'type']:
                source[field] = voice[field]
            
            source_id = source['id']
            # Add to tiktok_voices dict
            tiktok_voices[source_id] = source
            
print(len(tiktok_voices.keys()))
print('Done!')

In [ ]:
# And finally link with row data
count = 0

for raw_data in data:
    voice_id = raw_data['social']['profile']['id']
    if voice_id in tiktok_voices.keys():
        raw_data['voice'] = tiktok_voices[voice_id]
print('Done!')

In [ ]:
type(data)

In [ ]:
quarter = int(len(data) // 4)

data_1 = data[ : quarter]
data_2 = data[quarter : 2*quarter]
data_3 = data[2*quarter : 3*quarter]
data_4 = data[3*quarter : ]


In [ ]:
import pandas as pd
from pandas.io.json import json_normalize

merged = pd.json_normalize(data_4)

print('Done!')

In [ ]:
merged.to_pickle('BIG4.pickle')
print('Done!')

In [ ]:
################
################ YEYA !!!!!!!!
################
data = pd.read_pickle("BIG1.pickle")

In [ ]:
data.head()